In [19]:
import tensorflow as tf
import numpy as np

In [20]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import sklearn


map_polarity = {0:0, 4:2, 2:1}

def map_model_out(x):
    if x < 0.5:
        return 0
    elif x > 0.5:
        return 1
    return 2

print("\n\nSentiment 140:")
df = pd.read_csv("../trainModel/datasets/test.csv", encoding = "ISO-8859-1", names=['polarity', 'id', 'query', 'user', 'text'], index_col=2)
df = df.sample(frac=1)[:] # shuffle and truncate
df.drop(df.loc[df['polarity']==2].index, inplace=True)
print(set(df['polarity'].values))

# real_labels = df['polarity'].map(lambda x: map_polarity[x])
df['polarity'] = df['polarity'].apply(lambda x: 1 if x == 4 else 0)

test_data = df['text'].to_numpy()
test_label = df['polarity'].to_numpy()




Sentiment 140:
{0, 4}


In [24]:
with open("web-test/model.tflite", 'rb') as f :
    interpreter = tf.lite.Interpreter(model_content=f.read())
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(interpreter.get_input_details(),'\n', interpreter.get_output_details())

print(test_data[0])


predictions = []
for i, t in enumerate(test_data):
    interpreter.set_tensor(input_index, np.array([t]))
    interpreter.invoke()
    output = interpreter.tensor(output_index)
    predictions.append(output()[0][0])
    print(i, predictions[-1])

predictions = [map_model_out(x) for x in predictions]

print(sklearn.metrics.confusion_matrix(test_label, predictions))
print(list(predictions - test_label).count(0) / len(predictions))



2023-01-16 15:44:32.544957: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[{'name': 'serving_default_text_vectorization_input:0', 'index': 0, 'shape': array([1], dtype=int32), 'shape_signature': array([-1], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}] 
 [{'name': 'StatefulPartitionedCall_2:0', 'index': 125, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Time Warner cable is down again 3rd time since Memorial Day bummer!
0 -4.18075
1 -1.0766909
2 9.65689
3 -1.3482693
4 1.2517375
5 -5.209608
6 -0.59262097
7 -4.0717134
8 4.1789546
9 -0.8836008
10 -4.6240764
11 -2.808138
12 3.6600041
13 4.524804
14 -0.22586496
15 -2.5888495
16 